In [29]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

In [8]:
# def passi_luuka_feat_sel(data, measure='luca', p=1):
#     num_classes = int(np.max(data[:, -1]) + 1)  # Number of classes
#     num_samples = data.shape[0]             # Number of samples
#     num_features = data.shape[1] - 1        # Number of features
# 
#     data_old = data.copy()
# 
#     # Extract features and labels
#     data_v = data[:, :num_features]
#     data_c = data[:, num_features]
# 
#     # Initialize MinMaxScaler
#     scaler = MinMaxScaler()
# 
#     # Fit the scaler to the feature data and transform it
#     data_v = scaler.fit_transform(data_v)
# 
#     # Concatenate scaled features with labels
#     data = np.hstack((data_v, data_c.reshape(-1, 1)))
# 
#     # Forming idealvec using arithmetic mean
#     idealvec_s = np.zeros((num_classes, num_features))
#     for k in range(num_classes):
#         idealvec_s[k, :] = np.mean(data[data[:, -1] == k][:, :num_features], axis=0)
# 
#     # Sample data
#     datalearn_s = data[:, :num_features]
# 
#     # Initialize similarity matrix
#     sim = np.zeros((num_classes, num_samples, num_features))
# 
#     # Compute similarities
#     for i in range(num_classes):
#         mask = (data[:, -1] == i)
#         sim[i][mask] = (1 - np.abs(idealvec_s[i] ** p - datalearn_s[mask] ** p)) ** (1 / p)
# 
#     A = np.sum(sim[0], axis=0)
#     B = np.sum(sim[1], axis=0)
# 
#     Mu = A / (A + B)
# 
#     H = -(Mu * np.log(Mu) + (1 - Mu) * np.log(1 - Mu))
# 
#     return H


In [28]:
def passi_luuka_feat_sel(data, measure='luca', p=1):
    num_classes = int(np.max(data[:, -1]) + 1)  # Number of classes
    num_samples = data.shape[0]             # Number of samples
    num_features = data.shape[1] - 1        # Number of features

    data_old = data.copy()
#-------------------------------------------------
    # Extract features and labels
    data_v = data[:, :num_features]
    data_c = data[:, num_features]

    # Initialize MinMaxScaler
    scaler = MinMaxScaler()

    # Fit the scaler to the feature data and transform it
    data_v = scaler.fit_transform(data_v)
# # --------------------------------
#     data_v = data[:, :num_features]  # Extract first t columns
#     data_c = data[:, num_features ]  # Extract column t+1 as labels
#
#     mins_v = np.min(data_v, axis=0)  # Find minimum values along each column
#     Ones = np.ones_like(data_v)  # Create matrix of ones with same shape as data_v
#
#     # Perform matrix multiplication and addition to shift values
#     data_v = data_v + Ones @ np.diag(np.abs(mins_v))
# # ---------------------------------

    # Concatenate scaled features with labels
    data = np.hstack((data_v, data_c.reshape(-1, 1)))
    # Forming idealvec using arithmetic mean
    idealvec_s = np.zeros((num_classes, num_features))
    for k in range(num_classes):
        idealvec_s[k, :] = np.mean(data[data[:, -1] == k][:, :num_features], axis=0)

    # Sample data
    datalearn_s = data[:, :num_features]

    # Initialize similarity matrix
    sim = np.zeros((num_classes, num_samples, num_features))

    # Compute similarities
    for i in range(num_classes):
        mask = (data[:, -1] == i)
        sim[i][mask] = (1 - np.abs(idealvec_s[i] ** p - datalearn_s[mask] ** p)) ** (1 / p)

    delta = 1E-10
    sim[sim == 0] = delta
    sim[sim == 1] = 1 - delta

    sim = np.reshape(sim,(num_samples*num_classes,num_features))

    H = np.sum(-(sim * np.log2(sim) + (1-sim) * np.log2(1-sim)), axis=0)

    # ----------------
    # A = np.sum(sim[0], axis=0)
    # B = np.sum(sim[1], axis=0)
    #
    # Mu = A / (A + B)
    #
    # Mu_2 = B / (A+B)
    #
    # H = -(Mu * np.log2(Mu) + (Mu_2) * np.log2(Mu_2))
    # --------------------------

    # H = -(Mu * np.log(Mu) + (1 - Mu) * np.log(1 - Mu))

    return H

In [30]:
df = pd.read_csv("../../data/BinaryClassify/train_nsl_kdd_binary_encoded.csv")

In [31]:
data = np.array(df)

In [32]:
p_luuka_entropy = passi_luuka_feat_sel(data)

In [33]:
feature_entropy_values = {}

for feature_column in df.columns[:-1]:  # Exclude the target column
    idx = df.columns.get_loc(feature_column)
    feature_entropy_values[feature_column] = p_luuka_entropy[idx]

In [36]:
# Sort the results in descending order
sorted_results = sorted(feature_entropy_values.items(), key=lambda x: x[1],reverse=True)


In [37]:
sorted_results

[('dst_host_count', 92778.09323260994),
 ('dst_host_same_srv_rate', 81964.38259904082),
 ('dst_host_srv_count', 78752.93880708754),
 ('service', 75702.78023313169),
 ('dst_host_same_src_port_rate', 73535.63350375429),
 ('logged_in', 71002.86947704153),
 ('dst_host_serror_rate', 64978.2170593813),
 ('serror_rate', 64908.63186393407),
 ('srv_serror_rate', 64485.18895769182),
 ('dst_host_rerror_rate', 61598.199240626316),
 ('dst_host_srv_rerror_rate', 61511.38859457096),
 ('same_srv_rate', 61338.501010128726),
 ('dst_host_srv_serror_rate', 61241.31001468686),
 ('srv_rerror_rate', 61233.65694092055),
 ('rerror_rate', 61089.5458095035),
 ('protocol_type', 56392.80043503252),
 ('srv_diff_host_rate', 55511.3554640921),
 ('count', 54272.812186863666),
 ('flag', 43813.23923647417),
 ('dst_host_diff_srv_rate', 41596.58032041527),
 ('level', 38894.03041295156),
 ('srv_count', 33862.841075137585),
 ('diff_srv_rate', 32883.16836107924),
 ('dst_host_srv_diff_host_rate', 28134.506598883894),
 ('is_gu

In [38]:
# Create a DataFrame from the sorted results
result_df = pd.DataFrame(sorted_results, columns=['Feature', 'PassiLuukaEntropy'])

In [39]:
result_df

,Feature,PassiLuukaEntropy
0,dst_host_count,92778.093233
1,dst_host_same_srv_rate,81964.382599
2,dst_host_srv_count,78752.938807
3,service,75702.780233
4,dst_host_same_src_port_rate,73535.633504
5,logged_in,71002.869477
6,dst_host_serror_rate,64978.217059
7,serror_rate,64908.631864
8,srv_serror_rate,64485.188958
9,dst_host_rerror_rate,61598.199241


In [42]:
# Save the results to an Excel file
result_df.to_excel('passi_luuka_results.xlsx', index=False)

In [40]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,level,binaryoutcome
0,0,2,44,9,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,15,1
1,0,1,49,5,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.00,19,0
2,0,1,24,9,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,21,1
3,0,1,24,9,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,21,1
4,0,1,49,1,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,21,0


In [41]:
df.columns.get_loc('num_outbound_cmds')

19